In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [25]:
import sys
import collections
import cv2
import os
import math
import random
import pickle
import copy
import numpy as np
sys.path.append("/home/tanvikamble/MeronymNet-PyTorch/src")

In [26]:
import torch
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import DataLoader
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

In [27]:
from losses import BoxVAE_losses as loss
from components.AutoEncoder import GCNAutoEncoder
from components.Decoder import Decoder

In [28]:
import matplotlib.pyplot as plt

In [29]:
# from mask_generation import masked_sketch

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [31]:
colors = [(1, 0, 0),
          (0.737, 0.561, 0.561),
          (0.255, 0.412, 0.882),
          (0.545, 0.271, 0.0745),
          (0.98, 0.502, 0.447),
          (0.98, 0.643, 0.376),
          (0.18, 0.545, 0.341),
          (0.502, 0, 0.502),
          (0.627, 0.322, 0.176),
          (0.753, 0.753, 0.753),
          (0.529, 0.808, 0.922),
          (0.416, 0.353, 0.804),
          (0.439, 0.502, 0.565),
          (0.784, 0.302, 0.565),
          (0.867, 0.627, 0.867),
          (0, 1, 0.498),
          (0.275, 0.51, 0.706),
          (0.824, 0.706, 0.549),
          (0, 0.502, 0.502),
          (0.847, 0.749, 0.847),
          (1, 0.388, 0.278),
          (0.251, 0.878, 0.816),
          (0.933, 0.51, 0.933),
          (0.961, 0.871, 0.702)]
colors = (np.asarray(colors)*255)
canvas_size = 660

def plot_bbx(bbx):
    bbx = bbx*canvas_size
    canvas = np.ones((canvas_size,canvas_size,3), np.uint8) * 255
    for i, coord in enumerate(bbx):
        x_minp, y_minp,x_maxp , y_maxp= coord
        if [x_minp, y_minp,x_maxp , y_maxp]!=[0,0,0,0]:
            cv2.rectangle(canvas, (int(x_minp), int(y_minp)), (int(x_maxp) , int(y_maxp) ), colors[i], 6)
    return canvas

# def plot_bbx(bbx):
#     bbx = bbx*canvas_size
#     canvas = np.ones((canvas_size,canvas_size,3), np.uint8) * 255
#     for i, coord in enumerate(bbx):
#         x, y, w ,h = coord
#         if [x, y, w ,h]!=[0,0,0,0]:
#             cv2.rectangle(canvas, (int(x), int(y)), (int(x + w) , int(y + h) ), colors[i], 6)
#     return canvas


In [32]:
def inference(decoder, nodes, obj_class, latent_dims, batch_size):

    decoder.cuda()
    z_latent = torch.normal(torch.zeros([batch_size,latent_dims])).cuda()
    nodes = torch.reshape(nodes,(batch_size,decoder.num_nodes))
    obj_class = torch.reshape(obj_class,(batch_size, decoder.class_size))
    conditioned_z = torch.cat([nodes, z_latent],dim=-1)
    conditioned_z = torch.cat([obj_class, conditioned_z],dim=-1)
    
    x_bbx, x_lbl = decoder(conditioned_z)
    if x_lbl == None:
        return x_bbx,z_latent
    else:
        return x_bbx, x_lbl, z_latent

In [33]:
outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/X_test.np'
with open(outfile, 'rb') as pickle_file:
    X_test = pickle.load(pickle_file)

outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/adj_test.np'
with open(outfile, 'rb') as pickle_file:
    adj_test = pickle.load(pickle_file)
    
outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/class_v_test.np'
with open(outfile, 'rb') as pickle_file:
    class_v_test = pickle.load(pickle_file)

In [34]:
X_test[X_test<=0] = 0
X_test[X_test>=1] = 1

In [35]:
random.seed(100)
test_idx = np.random.randint(1,len(X_test),len(X_test))

In [36]:
batch_size = 128
seed = 345
torch.manual_seed(seed)
val_list = []
for idx, batch in enumerate(zip(copy.deepcopy(X_test[test_idx]),
                                copy.deepcopy(class_v_test[test_idx]), 
                                copy.deepcopy(adj_test[test_idx]))):
    edge_index, _ = dense_to_sparse(torch.from_numpy(batch[2]).cuda().float())
    val_list.append(Data(x = torch.from_numpy(batch[0]).cuda().float(),
                         y = torch.from_numpy(batch[1]).cuda().float(),
                         edge_index = edge_index
                                )
                     )
batch_val_loader = DataLoader(val_list, batch_size=batch_size)

In [37]:
del val_list


In [38]:
idx = 0
for data in batch_val_loader:
    idx+=1
    print(data.x.shape)
    if idx==3:
        break

torch.Size([3072, 5])
torch.Size([3072, 5])
torch.Size([3072, 5])


In [39]:
latent_dims = 64
batch_size = 128
num_nodes = 24
bbx_size = 4
num_classes = 10
label_shape = 1
nb_epochs = 250
klw = loss.frange_cycle_linear(nb_epochs)
learning_rate = 0.000065
hidden1 = 32
hidden2 = 16
hidden3 = 128
run_prefix = "Comparison-6"

In [40]:
import gc
gc.collect()

764580

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [42]:
summary_path = ('/home/tanvikamble/MeronymNet-PyTorch/src/runs/'+run_prefix+'/GCN-lr-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
write_tensorboard = True
if write_tensorboard:
    writer = SummaryWriter(summary_path)


In [43]:
#testing loop
model_path_1 = ('/home/tanvikamble/MeronymNet-PyTorch/src/model/'+'Reconstruction_without_Node_Loss_without_margin'+'/GCN-lr-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')

model_path_2 = ('/home/tanvikamble/MeronymNet-PyTorch/src/model/'+'Reconstruction_without_Node_Loss-2'+'/GCN-lr-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')



model_path_3 = ('/home/tanvikamble/MeronymNet-PyTorch/src/model/'+'Reconstruction_without_Node_Loss-3'+'/GCN-lr-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')

class_dict = ['cow', 'sheep', 'bird', 'person', 'cat', 'dog', 'horse', 'aeroplane',
              'motorbike', 'bicycle', 'car']
count_dict = {'cow':0, 'sheep':0, 'bird':0, 'person':0, 'cat':0, 'dog':0, 'horse':0,
              'aeroplane':0, 'motorbike':0, 'bicycle':0, 'car':0}
image_shape = [num_nodes, bbx_size]
global_step = 250000

vae_1 = GCNAutoEncoder(latent_dims,num_nodes,bbx_size,num_classes,label_shape,hidden1, hidden2, hidden3, dynamic_margin=False)
vae_1.load_state_dict(torch.load(model_path_1+ '/model_weights.pth'))

vae_2 = GCNAutoEncoder(latent_dims,num_nodes,bbx_size,num_classes,label_shape,hidden1, hidden2, hidden3, dynamic_margin=True)
vae_2.load_state_dict(torch.load(model_path_2+ '/model_weights.pth'))

vae_3 = GCNAutoEncoder(latent_dims,num_nodes,bbx_size,num_classes,label_shape,hidden1, hidden2, hidden3, dynamic_margin=True)
vae_3.load_state_dict(torch.load(model_path_3+ '/model_weights.pth'))


decoder_1 = vae_1.decoder
decoder_2 = vae_2.decoder
decoder_3= vae_3.decoder

for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data.cuda()
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    class_true = val_data.y
    val_batch_size = int(class_true.shape[0]/10)
    adj_true = val_data.edge_index
#     output = inference(decoder, label_true , class_true, latent_dims, val_batch_size)
    # First Model
    output_1 = vae_1(adj_true, node_data_true, label_true , class_true)
    node_data_pred_test_1 = output_1[0]
    # Second Model
    output_2 = vae_2(adj_true, node_data_true, label_true , class_true)
    node_data_pred_test_2 = output_2[0]
    # Third Model
    output_3 = vae_3(adj_true, node_data_true, label_true , class_true)
    node_data_pred_test_3 = output_3[0]
    
    if write_tensorboard:
        for j in range(int(len(node_data_true)/24)):
            
            obj_class = class_dict[int(np.argmax(class_true[10*j:10*(j+1)].detach().to('cpu').numpy()).tolist())]
            if count_dict[obj_class]<10:
                print(obj_class, count_dict[obj_class])
                image = plot_bbx(np.reshape((node_data_true[24*j:24*(j+1),1:]*label_true[24*j:24*(j+1)]).detach().to("cpu").numpy(),
                                            image_shape)).astype(float)/255
                # First output
                pred_image_1 = plot_bbx((node_data_pred_test_1[j]*label_true[24*j:24*(j+1)]).detach().to("cpu").numpy()).astype(float)/255
                
                # Second output
                pred_image_2 = plot_bbx((node_data_pred_test_2[j]*label_true[24*j:24*(j+1)]).detach().to("cpu").numpy()).astype(float)/255
                
                 # Third output
                pred_image_3 = plot_bbx((node_data_pred_test_3[j]*label_true[24*j:24*(j+1)]).detach().to("cpu").numpy()).astype(float)/255

#                 writer.add_image('test_result/images/'+obj_class+'/'+str(j)+'-input/', image, global_step, dataformats='HWC')  
#                 writer.add_image('test_result/images/'+obj_class+'/'+str(j)+'-generated/', pred_image, global_step, dataformats='HWC')
                # create figure
                fig = plt.figure(figsize=(15, 15))

                # setting values to rows and column variables
                rows = 2
                columns = 2
                # Adds a subplot at the 1st position
                fig.add_subplot(rows, columns, 1)

                # showing image
                plt.imshow(image)
                plt.axis('off')
                plt.title("Ground Truth")
                
                # Adds a subplot at the 3rd position
                fig.add_subplot(rows, columns, 2)

                # showing image
                plt.imshow(pred_image_1)
                plt.axis('off')
                plt.title("Without Margin")

                # Adds a subplot at the 2nd position
                fig.add_subplot(rows, columns, 3)

                # showing image
                plt.imshow(pred_image_2)
                plt.axis('off')
                plt.title("With Margin in Loss")
                
                 # Adds a subplot at the 2nd position
                fig.add_subplot(rows, columns, 4)

                # showing image
                plt.imshow(pred_image_3)
                plt.axis('off')
                plt.title("With Margin in Loss (sum instead of mean)")

                

                temp=plt.gcf()
#                 plt.show()
                print('Hello1')
                writer.add_figure('test_result/plots/'+obj_class+'/'+str(j),temp, global_step=global_step, close=True, walltime=None)
                print('Hello2')
                count_dict[obj_class]+=1

writer.flush()
writer.close()
    


person 0
Hello1
Hello2
cat 0
Hello1
Hello2
motorbike 0
Hello1
Hello2
horse 0
Hello1
Hello2
bicycle 0
Hello1
Hello2
dog 0
Hello1
Hello2
dog 1
Hello1
Hello2
cat 1
Hello1
Hello2
cat 2
Hello1
Hello2
motorbike 1
Hello1
Hello2
dog 2
Hello1
Hello2
bicycle 1
Hello1
Hello2
bird 0
Hello1
Hello2
dog 3
Hello1
Hello2
sheep 0
Hello1
Hello2
motorbike 2
Hello1
Hello2
aeroplane 0
Hello1
Hello2
motorbike 3
Hello1
Hello2
cat 3
Hello1
Hello2
sheep 1
Hello1
Hello2
horse 1
Hello1
Hello2
aeroplane 1
Hello1
Hello2
person 1
Hello1
Hello2
aeroplane 2
Hello1
Hello2
motorbike 4
Hello1
Hello2
aeroplane 3
Hello1
Hello2
aeroplane 4
Hello1
Hello2
bicycle 2
Hello1
Hello2
dog 4
Hello1
Hello2
cat 4
Hello1
Hello2
sheep 2
Hello1
Hello2
person 2
Hello1
Hello2
bird 1
Hello1
Hello2
aeroplane 5
Hello1
Hello2
cat 5
Hello1
Hello2
dog 5
Hello1
Hello2
dog 6
Hello1
Hello2
cat 6
Hello1
Hello2
cat 7
Hello1
Hello2
motorbike 5
Hello1
Hello2
bicycle 3
Hello1
Hello2
cat 8
Hello1
Hello2
bicycle 4
Hello1
Hello2
cat 9
Hello1
Hello2
aeropla